In [7]:
import sys
sys.path.append('/Users/celia/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/celia/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import scipy.io as scio
import bandit_preprocessing as bp
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline

## extract_frames():  function to get frames based on 1-3 conditions
#### Input variables:
        df = feature matrix
        cond(n)_name = string containing column name (i.e. Reward, Switch) 
        cond(n) = desired identity (0,1)
#### Output variables:
        frames = (num_Trials x 2) matrix containing frame # for center poke and decision poke for each trial

In [2]:
def extract_frames(df, cond1_name, cond1=False, cond2_name=False,cond2=False, cond3_name=False,
                   cond3=False, cond1_ops= '=', cond2_ops = '=', cond3_ops = '='):
                    # First define function so it can take multiple conditions
    import operator
    
    # set up operator dictionary
    ops = {'>': operator.gt,
       '<': operator.lt,
       '>=': operator.ge,
       '<=': operator.le,
       '=': operator.eq}
    
    if type(cond3_name)==str:
        frames_c = (df[((ops[cond1_ops](df[cond1_name],cond1)) 
                    & (ops[cond2_ops](df[cond2_name], cond2))
                    & (ops[cond3_ops](df[cond3_name],cond3)))]['center_frame'])
        frames_d = (df[((ops[cond1_ops](df[cond1_name],cond1)) 
                    & (ops[cond2_ops](df[cond2_name], cond2))
                    & (ops[cond3_ops](df[cond3_name],cond3)))]['decision_frame'])
        frames = np.column_stack((frames_c, frames_d))
        return frames
    
    elif type(cond2_name)==str:
        frames_c = (df[((df[cond1_name] == cond1) 
                    & (df[cond2_name] == cond2))]['center_frame'])
        frames_d = (df[((df[cond1_name] == cond1) 
                    & (df[cond2_name] == cond2))]['decision_frame'])
        frames = np.column_stack((frames_c, frames_d))
        return frames
    
    else:
        frames_c =(df[(df[cond1_name] == cond1)]['center_frame'])
        frames_d =(df[(df[cond1_name] == cond1)]['decision_frame'])
        frames = np.column_stack((frames_c, frames_d))
        return frames

In [4]:
def simple_extract_frames(df, cond_name, cond_id=False):
    frames_center = df[df[cond_name]==cond_id]['center_frame']
    frames_decision = df[df[cond_name]==cond_id]['decision_frame']
    frames = np.column_stack((frames_center, frames_decision))
    return frames

In [13]:
'''Initialize paths and import data'''

record_path = '/Users/celia/GitHub/mouse_bandit/session_record.csv' # session record of behavioral data
ca_data_path = '/Volumes/Neurobio/MICROSCOPE/Celia/data/imaging/Shay/q43_03202017_bandit_8020_local/q43_03202017_neuron_master.mat'

record = pd.read_csv(record_path,index_col=0) # session record is panda df
ca_data = scio.loadmat(ca_data_path,squeeze_me = True, struct_as_record = False)
df = ca_data['neuron_results'] # set neuron variable to just contain neuron_results structure


'''Specify mouse and session of interest'''

session_name  = '03202017_Q43'
mouse_id = 'Q43'

record[record['Session ID'] == session_name] # take only rows from record that match session name


'''Load in trial data from specific session'''

columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked',
           'Right Reward Prob','Left Reward Prob','Reward Given',
          'center_frame','decision_frame']

root_dir = '/Users/celia/GitHub/mouse_bandit/data/trial_data'

full_name = session_name + '_trials.csv'

path_name = os.path.join(root_dir,full_name)

trial_df = pd.read_csv(path_name,names=columns)

In [14]:
trial_df.head(5)

,Elapsed Time (s),Since last trial (s),Trial Duration (s),Port Poked,Right Reward Prob,Left Reward Prob,Reward Given,center_frame,decision_frame
0,35.902,22.129,0.621,1,0.2,0.8,0,1460,1479
1,40.081,1.949,0.522,2,0.2,0.8,1,1588,1604
2,41.838,1.377,0.380,2,0.2,0.8,0,1646,1657
3,58.512,1.438,0.378,1,0.2,0.8,0,2146,2157
4,62.833,2.381,0.491,2,0.2,0.8,1,2272,2287


In [29]:
test = simple_extract_frames(trial_df, 'Reward Given', 1)
trial_df['center_frame']test[0]

SyntaxError: invalid syntax (<ipython-input-29-0b6b57fc5b17>, line 2)

In [27]:
test2 = simple_extract_frames(test, 'Port Poked', 1)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices